---
# **Property Assessment CCAO:** FIN 550 Final Project

This R script contains the full workflow to:
   - Load and process (ETL) Cook County property datasets
   - Perform Exploratory Data Analysis (EDA)
   - Develop and apply models for predicting residential property values
 
The script produces the final 'assessed_value.csv' file for project submission.


---
### **Extract, Transform, Load**

##### Install & Load Required Packages

In [180]:
# Install and load required packages

# Function to install packages if not already installed
install_if_missing <- function(package) {
  if (!require(package, character.only = TRUE, quietly = TRUE)) {
    install.packages(package, dependencies = TRUE)
  }
}

# Data manipulation and ETL
install_if_missing("readr")
install_if_missing("dplyr")
install_if_missing("tidyr")
install_if_missing("data.table")

# EDA and visualization
install_if_missing("ggplot2")
install_if_missing("corrplot")
install_if_missing("GGally")
install_if_missing("skimr")

# Modeling - Linear Models
install_if_missing("caret")
install_if_missing("glmnet")

# Modeling - Tree-based methods
install_if_missing("rpart")
install_if_missing("rpart.plot")
install_if_missing("randomForest")
install_if_missing("xgboost")

# Model evaluation and tuning
install_if_missing("Metrics")
install_if_missing("MLmetrics")

# Missing data imputation
install_if_missing("mice")

# Load libraries
library(readr)       # Reading CSV files
library(dplyr)       # Data manipulation
library(tidyr)       # Data tidying
library(data.table)  # Fast data manipulation

library(ggplot2)     # Visualization
library(corrplot)    # Correlation plots
library(GGally)      # Pairwise plots
library(skimr)       # Summary statistics

library(caret)       # Model training and evaluation
library(glmnet)      # Regularized regression (Ridge, Lasso, Elastic Net)

library(rpart)       # Decision trees
library(rpart.plot)  # Decision tree visualization
library(randomForest) # Random Forest
library(xgboost)     # XGBoost

library(Metrics)     # Model evaluation metrics
library(MLmetrics)   # Additional ML metrics

library(mice)        # Multiple imputation by chained equations

# Set random seed for reproducibility
set.seed(550)

cat("All required packages loaded successfully.\n")


All required packages loaded successfully.


##### Load the Datasets

In [181]:
# Load the datasets
historic_data <- read_csv('data/historic_property_data.csv')
predict_set <- read_csv('data/predict_property_data.csv')

# Display basic information about the datasets
cat("Historic Data Shape:", dim(historic_data), "\n")
cat("Predict Set Shape:", dim(predict_set), "\n")


Rows: 50000 Columns: 63
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): meta_cdu, meta_deed_type, geo_property_city, geo_property_zip, geo...
dbl (52): sale_price, meta_class, meta_town_code, meta_nbhd, meta_certified_...
lgl  (3): ind_large_home, ind_garage, ind_arms_length

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 10000 Columns: 63
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): meta_cdu, meta_deed_type, geo_property_city, geo_property_zip, geo...
dbl (52): pid, meta_class, meta_town_code, meta_nbhd, meta_certified_est_bld...
lgl  (3): ind_large_home, ind_garage, ind_arms_length

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Historic Data Shape: 50000 63 
Predict Set Shape: 10000 63 


##### Observe Missing Values

In [182]:
# Check for missing values in historic_data
cat("\n=== Missing Values in Historic Data ===\n")
missing_historic <- colSums(is.na(historic_data))
missing_historic_pct <- (missing_historic / nrow(historic_data)) * 100

# Display columns with missing values
missing_historic_df <- data.frame(
  Variable = names(missing_historic),
  Missing_Count = missing_historic,
  Missing_Percent = round(missing_historic_pct, 2)
) %>%
  filter(Missing_Count > 0) %>%
  arrange(desc(Missing_Count))

if(nrow(missing_historic_df) > 0) {
  print(missing_historic_df, row.names = FALSE)
  cat("\nTotal columns with missing values:", nrow(missing_historic_df), "\n")
} else {
  cat("No missing values found in historic data.\n")
}

# Check for missing values in predict_set
cat("\n=== Missing Values in Predict Set ===\n")
missing_predict <- colSums(is.na(predict_set))
missing_predict_pct <- (missing_predict / nrow(predict_set)) * 100

# Display columns with missing values
missing_predict_df <- data.frame(
  Variable = names(missing_predict),
  Missing_Count = missing_predict,
  Missing_Percent = round(missing_predict_pct, 2)
) %>%
  filter(Missing_Count > 0) %>%
  arrange(desc(Missing_Count))

if(nrow(missing_predict_df) > 0) {
  print(missing_predict_df, row.names = FALSE)
  cat("\nTotal columns with missing values:", nrow(missing_predict_df), "\n")
} else {
  cat("No missing values found in predict set.\n")
}

# Summary of missing values
cat("\n=== Summary ===\n")
cat("Historic Data - Total missing values:", sum(missing_historic), "\n")
cat("Predict Set - Total missing values:", sum(missing_predict), "\n")



=== Missing Values in Historic Data ===
                    Variable Missing_Count Missing_Percent
             char_renovation         49741           99.48
                    meta_cdu         47172           94.34
                   char_apts         43093           86.19
                  char_porch         40725           81.45
             char_attic_fnsh         33453           66.91
                char_tp_dsgn         27173           54.35
                char_tp_plan         14394           28.79
              char_gar1_area          7090           14.18
              char_gar1_cnst          7088           14.18
               char_gar1_att          7088           14.18
                    geo_fips          1103            2.21
            geo_municipality          1103            2.21
                  char_oheat           172            0.34
               geo_tract_pop           162            0.32
              geo_white_perc           162            0.32
              g

##### Missing Values by Data Type *(Nominal, Ordinal, Integer, Continuous, Booleen)*

In [183]:
# Analyze missing values by codebook-based data type
cat("\n=== Missing Values by Data Type (Codebook-Informed) ===\n")

# Load codebook if not already loaded
if(!exists("codebook_df")) {
  codebook_df <- read.csv("data/codebook.csv", stringsAsFactors = FALSE)
}

# Create a lookup table: name -> type (using codebook var_type and var_data_type for more granularity)
get_codebook_types <- function() {
  # Try to get a clean set of variable names mapping to their type and data type
  out <- codebook_df
  # All possible codebook variable names (columns in datasets) to possible types
  # Favor var_name_standard (it's primary for standardization)
  out <- out[!is.na(out$var_name_standard) & nzchar(out$var_name_standard), ]
  var_type_map <- setNames(out$var_type, out$var_name_standard)
  var_data_type_map <- setNames(out$var_data_type, out$var_name_standard)
  return(list(type=var_type_map, datatype=var_data_type_map))
}

cb_types <- get_codebook_types()

# Function to group codebook data types into our analysis buckets
codebook_col_type <- function(var, cb_types) {
  # var: column name
  type1 <- cb_types$type[[var]]
  type2 <- cb_types$datatype[[var]]
  if (is.null(type1) && is.null(type2)) return("Unknown")
  
  # Use var_type, then var_data_type to classify
  # Explicit logic:
  # - boolean: logical
  # - categorical: categorical (nominal or ordinal handled together; deeper split possible using var_notes, but not here)
  # - numeric: numeric (includes continuous, integer)
  # - everything else: character/fallback
  
  # Map codebook types to user buckets:
  # If it's explicit logical in codebook, call it 'Boolean'
  if(!is.null(type2) && tolower(type2) == "logical") return("Boolean")
  if(!is.null(type1) && tolower(type1) == "ind") return("Boolean")
  
  # Categorical can be nominal or ordinal: if type2 is "categorical" or type1 is "char"
  if(!is.null(type2) && tolower(type2) == "categorical") return("Categorical")
  if(!is.null(type1) && type1 %in% c("char", "meta", "geo", "econ")) {
    # meta/geo/econ might include character or codes, but check var_data_type for "categorical"
    if(!is.null(type2) && tolower(type2) == "categorical") return("Categorical")
  }
  
  # Numeric (continuous, integer)
  if(!is.null(type2) && tolower(type2) == "numeric") return("Numeric")
  
  # If anything says 'character'
  if(!is.null(type2) && tolower(type2) == "character") return("Categorical")
  
  # Fallback: Unknown
  return("Unknown")
}

# Calculate missing by codebook-based type
analyze_missing_by_codebook_type <- function(data, dataset_name, cb_types) {
  cat("\n", dataset_name, ":\n", sep = "")
  cols_with_missing <- names(data)[colSums(is.na(data)) > 0]
  if(length(cols_with_missing) == 0) {
    cat("  No missing values found.\n")
    return(NULL)
  }
  # Assign each col to codebook type
  cb_type_per_col <- sapply(cols_with_missing, codebook_col_type, cb_types=cb_types)
  types_for_summary <- c("Numeric", "Categorical", "Boolean", "Unknown")
  total_missing <- sum(is.na(data))
  missing_vals_by_type <- sapply(types_for_summary, function(btype) {
    these_cols <- cols_with_missing[cb_type_per_col == btype]
    if(length(these_cols)>0) sum(is.na(data[these_cols])) else 0
  })
  # Calculate percentages
  missing_pct_by_type <- if(total_missing>0) missing_vals_by_type/total_missing*100 else rep(0, length(missing_vals_by_type))
  names(missing_vals_by_type) <- types_for_summary
  names(missing_pct_by_type) <- types_for_summary
  
  # Print results
  for(type_str in types_for_summary) {
    cat(sprintf("  %-11s: %.2f%% (%d missing values)\n", 
                paste0(type_str, if(type_str=="Boolean") "" else ""),
                round(missing_pct_by_type[type_str], 2),
                missing_vals_by_type[type_str]))
  }
  cat(sprintf("  Total:       100.00%% (%d missing values)\n", total_missing))
}

# Apply to both datasets
analyze_missing_by_codebook_type(historic_data, "Historic Data", cb_types)
analyze_missing_by_codebook_type(predict_set, "Predict Set", cb_types)



=== Missing Values by Data Type (Codebook-Informed) ===

Historic Data:
  Numeric    : 0.52% (1483 missing values)
  Categorical: 99.24% (280338 missing values)
  Boolean    : 0.24% (672 missing values)
  Unknown    : 0.00% (0 missing values)
  Total:       100.00% (282493 missing values)

Predict Set:
  Numeric    : 0.01% (3 missing values)
  Categorical: 99.99% (55579 missing values)
  Boolean    : 0.01% (3 missing values)
  Unknown    : 0.00% (0 missing values)
  Total:       100.00% (55585 missing values)


##### Missing Value Handling *(Drop High-Missing Columns)*

- Drop any column with >50% missing values from both datasets

In [184]:
# Identify columns with >50% missing in historic data
high_missing_historic <- names(missing_historic_pct[missing_historic_pct > 50])
cat("\nColumns with >50% missing in historic data:", length(high_missing_historic), "\n")
if(length(high_missing_historic) > 0) {
  print(high_missing_historic)
}

# Identify columns with >50% missing in predict set
high_missing_predict <- names(missing_predict_pct[missing_predict_pct > 50])
cat("\nColumns with >50% missing in predict set:", length(high_missing_predict), "\n")
if(length(high_missing_predict) > 0) {
  print(high_missing_predict)
}

# Get union of columns to drop (present in either dataset)
cols_to_drop <- unique(c(high_missing_historic, high_missing_predict))
cat("\nTotal unique columns to drop:", length(cols_to_drop), "\n")

# Drop high-missing columns from both datasets
if(length(cols_to_drop) > 0) {
  cols_to_drop_historic <- intersect(cols_to_drop, names(historic_data))
  cols_to_drop_predict <- intersect(cols_to_drop, names(predict_set))
  
  if(length(cols_to_drop_historic) > 0) {
    historic_data <- historic_data %>% select(-all_of(cols_to_drop_historic))
    cat("Dropped", length(cols_to_drop_historic), "columns from historic data\n")
  }
  
  if(length(cols_to_drop_predict) > 0) {
    predict_set <- predict_set %>% select(-all_of(cols_to_drop_predict))
    cat("Dropped", length(cols_to_drop_predict), "columns from predict set\n")
  }
}

# Check remaining missing values
remaining_missing_historic <- sum(is.na(historic_data))
remaining_missing_predict <- sum(is.na(predict_set))

cat("\nRemaining missing values in historic data:", remaining_missing_historic, "\n")
cat("Remaining missing values in predict set:", remaining_missing_predict, "\n")


Columns with >50% missing in historic data: 6 
[1] "meta_cdu"        "char_apts"       "char_tp_dsgn"    "char_attic_fnsh"
[5] "char_renovation" "char_porch"     

Columns with >50% missing in predict set: 6 
[1] "meta_cdu"        "char_apts"       "char_tp_dsgn"    "char_attic_fnsh"
[5] "char_renovation" "char_porch"     

Total unique columns to drop: 6 
Dropped 6 columns from historic data
Dropped 6 columns from predict set

Remaining missing values in historic data: 41136 
Remaining missing values in predict set: 7319 


##### Missing Value Handling *(Analysis & Decision Support)*

- Analyze missing data patterns to decide on imputation strategy

In [185]:
# Function to analyze missing values by type, using CODEBOOK LOGIC (not R's class())
analyze_missing_by_codebook_type_detailed <- function(data, dataset_name, cb_types) {
  cat("Analysis for", dataset_name, "\n")
  
  # Find columns with missing values
  cols_with_missing <- names(data)[colSums(is.na(data)) > 0]
  
  if(length(cols_with_missing) == 0) {
    cat("No missing values found.\n")
    return(NULL)
  }
  
  # Assign codebook-based type to each column with missing values
  cb_types_col <- sapply(
    cols_with_missing, 
    function(var) codebook_col_type(var, cb_types), 
    USE.NAMES = FALSE
  )
  
  # Create detailed analysis dataframe
  missing_analysis <- data.frame(
    Variable = cols_with_missing,
    Codebook_Type = cb_types_col,
    Missing_Count = colSums(is.na(data[cols_with_missing])),
    Total_Rows = nrow(data),
    stringsAsFactors = FALSE
  ) %>%
    mutate(
      Missing_Percent = round((Missing_Count / Total_Rows) * 100, 2),
      Data_Category = factor(Codebook_Type, 
                             levels = c("Numeric", "Categorical", "Boolean", "Unknown"))
    ) %>%
    arrange(desc(Missing_Percent))
  
  # Analyze distributions for each codebook-based type
  for(type_lbl in c("Categorical", "Numeric", "Boolean", "Unknown")) {
    n_vars <- sum(missing_analysis$Data_Category == type_lbl)
    cat(sprintf("\n--- %s Variable Distributions ---\n", type_lbl))
    these_vars <- missing_analysis %>% filter(Data_Category == type_lbl) %>% pull(Variable)
    if(length(these_vars) > 0) {
      for(var in these_vars) {
        cat(var, "(", missing_analysis$Missing_Percent[missing_analysis$Variable == var], "% missing)\n", sep="")
      }
    } else {
      cat(sprintf("No %s variables with missing values.\n", tolower(type_lbl)))
    }
  }
  
  return(missing_analysis)
}

# Analyze both datasets using codebook-derived types
missing_analysis_historic <- analyze_missing_by_codebook_type_detailed(historic_data, "Historic Data", cb_types)
cat("\n")
missing_analysis_predict <- analyze_missing_by_codebook_type_detailed(predict_set, "Predict Set", cb_types)

# Row-wise missing analysis (using codebook logic for consistent col selection)
cat("\n=== Row-wise Missing Value Analysis ===\n")

analyze_row_completeness_cb <- function(data, dataset_name, missing_analysis_ref = NULL, type_target = "Categorical") {
  cat("\n--- Row completeness for", dataset_name, "---\n")
  
  # Count missing values per row
  missing_per_row <- rowSums(is.na(data))
  
  cat("Rows with 0 missing values:", sum(missing_per_row == 0), 
      sprintf("(%.2f%%)", sum(missing_per_row == 0) / nrow(data) * 100), "\n")
  cat("Rows with 1-2 missing values:", sum(missing_per_row >= 1 & missing_per_row <= 2), 
      sprintf("(%.2f%%)", sum(missing_per_row >= 1 & missing_per_row <= 2) / nrow(data) * 100), "\n")
  cat("Rows with 3-5 missing values:", sum(missing_per_row >= 3 & missing_per_row <= 5), 
      sprintf("(%.2f%%)", sum(missing_per_row >= 3 & missing_per_row <= 5) / nrow(data) * 100), "\n")
  cat("Rows with >5 missing values:", sum(missing_per_row > 5), 
      sprintf("(%.2f%%)", sum(missing_per_row > 5) / nrow(data) * 100), "\n")
  
  # If we have type-targeted analysis (e.g. "Categorical", "Boolean"), show effect of dropping those rows
  if(!is.null(missing_analysis_ref)) {
    type_cols <- missing_analysis_ref %>% 
      filter(Data_Category == type_target) %>% 
      pull(Variable)
    type_cols <- intersect(type_cols, names(data))
    
    if(length(type_cols) > 0) {
      rows_with_type_missing <- rowSums(is.na(data[type_cols])) > 0
      cat(sprintf("\nRows with missing %s values: %d (%.2f%% of dataset)\n",
                  tolower(type_target), sum(rows_with_type_missing), 
                  sum(rows_with_type_missing) / nrow(data) * 100))
      cat(sprintf("Data remaining if these rows dropped: %d (%.2f%% retention)\n",
                  nrow(data) - sum(rows_with_type_missing),
                  (nrow(data) - sum(rows_with_type_missing)) / nrow(data) * 100))
    }
  }
  
  return(missing_per_row)
}

row_missing_historic <- analyze_row_completeness_cb(historic_data, "Historic Data", missing_analysis_historic, "Categorical")
row_missing_predict <- analyze_row_completeness_cb(predict_set, "Predict Set", missing_analysis_predict, "Categorical")

Analysis for Historic Data 

--- Categorical Variable Distributions ---
char_tp_plan(28.79% missing)
char_gar1_cnst(14.18% missing)
char_gar1_att(14.18% missing)
char_gar1_area(14.18% missing)
geo_fips(2.21% missing)
geo_municipality(2.21% missing)
char_oheat(0.34% missing)
geo_school_elem_district(0.32% missing)
geo_school_hs_district(0.32% missing)
geo_property_city(0.26% missing)
geo_property_zip(0.26% missing)
char_cnst_qlty(0.11% missing)
char_ext_wall(0.05% missing)
char_roof_cnst(0.05% missing)
char_bsmt(0.05% missing)
char_bsmt_fin(0.05% missing)
char_heat(0.05% missing)
char_air(0.05% missing)
char_attic_type(0.05% missing)
char_site(0.05% missing)
char_gar1_size(0.05% missing)
char_repair_cnd(0.05% missing)
char_use(0.05% missing)
char_type_resd(0.05% missing)

--- Numeric Variable Distributions ---
geo_tract_pop(0.32% missing)
geo_white_perc(0.32% missing)
geo_black_perc(0.32% missing)
geo_asian_perc(0.32% missing)
geo_his_perc(0.32% missing)
geo_other_perc(0.32% missing)
ge

##### Missing Value Handling *(MICE Protocol)*

- **MICE Imputation** — Apply multivariate imputation by chained equations to remaining missing values:

   - ***PMM*** *(Predictive Mean Matching)* *{Numeric variables}*: Predicts missing values by matching to observed values with similar predicted means, preserving the original distribution
   - ***CART*** *(Classification and Regression Trees)* *{Categorical Variables}*: Uses decision tree models to predict missing categories based on other variables
   - ***LogReg*** *(Logistic Regression)* *{Logical/Binary Variables}*: Models the probability of TRUE/FALSE outcomes using logistic regression

 - MICE preserves relationships between variables and maintains distributional properties

In [186]:
# =====================================================
# PRE-IMPUTATION: CONVERT CATEGORICALS TO FACTORS
# =====================================================

cat("\n=== Converting Categorical Variables to Factors ===\n")
flush.console()

# Get codebook types
cb_types <- get_codebook_types()

# Convert categorical columns in historic_data
for(col in names(historic_data)) {
  # Skip if column doesn't exist in codebook
  if(!col %in% names(cb_types$type)) {
    next
  }
  
  col_type <- codebook_col_type(col, cb_types)
  
  if(col_type == "Categorical" && !is.factor(historic_data[[col]])) {
    historic_data[[col]] <- as.factor(historic_data[[col]])
    cat("  Converted historic_data$", col, " to factor (", 
        length(levels(historic_data[[col]])), " levels)\n", sep = "")
  } else if(col_type == "Boolean" && !is.logical(historic_data[[col]])) {
    historic_data[[col]] <- as.logical(historic_data[[col]])
    cat("  Converted historic_data$", col, " to logical\n", sep = "")
  }
}

# Convert categorical columns in predict_set
for(col in names(predict_set)) {
  # Skip if column doesn't exist in codebook
  if(!col %in% names(cb_types$type)) {
    next
  }
  
  col_type <- codebook_col_type(col, cb_types)
  
  if(col_type == "Categorical" && !is.factor(predict_set[[col]])) {
    predict_set[[col]] <- as.factor(predict_set[[col]])
    cat("  Converted predict_set$", col, " to factor (", 
        length(levels(predict_set[[col]])), " levels)\n", sep = "")
  } else if(col_type == "Boolean" && !is.logical(predict_set[[col]])) {
    predict_set[[col]] <- as.logical(predict_set[[col]])
    cat("  Converted predict_set$", col, " to logical\n", sep = "")
  }
}

cat("Categorical conversion complete.\n")
cat(sprintf("Timestamp: %s\n", Sys.time()))
flush.console()


=== Converting Categorical Variables to Factors ===
  Converted historic_data$meta_class to factor (14 levels)
  Converted historic_data$meta_town_code to factor (38 levels)
  Converted historic_data$meta_nbhd to factor (845 levels)
  Converted historic_data$meta_deed_type to factor (3 levels)
  Converted historic_data$char_ext_wall to factor (4 levels)
  Converted historic_data$char_roof_cnst to factor (6 levels)
  Converted historic_data$char_bsmt to factor (4 levels)
  Converted historic_data$char_bsmt_fin to factor (3 levels)
  Converted historic_data$char_heat to factor (4 levels)
  Converted historic_data$char_oheat to factor (2 levels)
  Converted historic_data$char_air to factor (2 levels)
  Converted historic_data$char_attic_type to factor (3 levels)
  Converted historic_data$char_tp_plan to factor (2 levels)
  Converted historic_data$char_cnst_qlty to factor (2 levels)
  Converted historic_data$char_site to factor (3 levels)
  Converted historic_data$char_gar1_size to factor

In [ ]:
# =====================================================
# MICE IMPUTATION
# =====================================================

cat("\n[CHECKPOINT] About to check if imputation needed...\n")
flush.console()

# Impute remaining missing values using MICE if any remain
if(remaining_missing_historic > 0 || remaining_missing_predict > 0) {
  cat("\n=== Imputing Missing Values with MICE ===\n")
  cat(sprintf("Timestamp: %s\n", Sys.time()))
  flush.console()

  set.seed(550)

  # Impute historic data
  if(remaining_missing_historic > 0) {
    cat("\n--- Processing Historic Data ---\n")
    cat(sprintf("Timestamp: %s\n", Sys.time()))
    flush.console()
    
    cols_with_missing_historic <- names(historic_data)[colSums(is.na(historic_data)) > 0]
    
    cat(sprintf("[PROGRESS] Found %d columns with missing values\n", length(cols_with_missing_historic)))
    flush.console()
    
    # Identify numeric columns with missing values for statistics tracking
    numeric_missing_historic <- cols_with_missing_historic[
      sapply(cols_with_missing_historic, function(col) {
        col %in% names(cb_types$type) && codebook_col_type(col, cb_types) == "Numeric"
      })
    ]
    
    # Identify categorical columns with missing values for distribution tracking
    categorical_missing_historic <- cols_with_missing_historic[
      sapply(cols_with_missing_historic, function(col) {
        col %in% names(cb_types$type) && codebook_col_type(col, cb_types) == "Categorical"
      })
    ]
    
    # Identify boolean columns with missing values for distribution tracking
    boolean_missing_historic <- cols_with_missing_historic[
      sapply(cols_with_missing_historic, function(col) {
        col %in% names(cb_types$type) && codebook_col_type(col, cb_types) == "Boolean"
      })
    ]

    cat(sprintf("[PROGRESS] Numeric: %d, Categorical: %d, Boolean: %d\n", 
                length(numeric_missing_historic), 
                length(categorical_missing_historic),
                length(boolean_missing_historic)))
    flush.console()

    if(length(numeric_missing_historic) > 0) {
      original_stats_historic <- data.frame(
        Variable = numeric_missing_historic,
        Original_Mean = sapply(historic_data[numeric_missing_historic], mean, na.rm = TRUE),
        Original_Median = sapply(historic_data[numeric_missing_historic], median, na.rm = TRUE),
        Original_Variance = sapply(historic_data[numeric_missing_historic], var, na.rm = TRUE),
        stringsAsFactors = FALSE
      )
    }
    
    # Store original categorical distributions
    if(length(categorical_missing_historic) > 0) {
      original_cat_dist_historic <- list()
      for(col in categorical_missing_historic) {
        original_cat_dist_historic[[col]] <- prop.table(table(historic_data[[col]], useNA = "no"))
      }
    }
    
    # Store original boolean distributions
    if(length(boolean_missing_historic) > 0) {
      original_bool_dist_historic <- list()
      for(col in boolean_missing_historic) {
        original_bool_dist_historic[[col]] <- prop.table(table(historic_data[[col]], useNA = "no"))
      }
    }

    cat("Variables with missing values:", length(cols_with_missing_historic), "\n")
    
    # Diagnostic: Analyze imputation complexity
    cat("\n=== Imputation Complexity Analysis (Historic Data) ===\n")
    for(col in cols_with_missing_historic) {
      n_missing <- sum(is.na(historic_data[[col]]))
      n_unique <- length(unique(historic_data[[col]][!is.na(historic_data[[col]])]))
      col_type <- class(historic_data[[col]])[1]
      
      cat(sprintf("%s: %d missing, %d unique values, type=%s\n",
                  col, n_missing, n_unique, col_type))
    }
    flush.console()
    
    cat("\n[PROGRESS] Creating predictor matrix with quickpred()...\n")
    cat(sprintf("Timestamp: %s\n", Sys.time()))
    flush.console()
    
    pred_matrix_historic <- quickpred(historic_data, mincor = 0.3, minpuc = 0.5)
    
    cat("[PROGRESS] Predictor matrix created successfully\n")
    cat(sprintf("Timestamp: %s\n", Sys.time()))
    flush.console()
    
    impute_methods_historic <- make.method(historic_data)

    # Assign imputation methods based on codebook data types
    cat("\n=== Assigning Imputation Methods (Historic Data) ===\n")
    flush.console()
    
    method_counts <- list(pmm = 0, cart = 0, polyreg = 0, sample = 0, logreg = 0)
    
    for(col in names(historic_data)) {
      if(sum(is.na(historic_data[[col]])) > 0) {
        # Skip if column doesn't exist in codebook
        if(!col %in% names(cb_types$type)) {
          cat(sprintf("  %s: skipped (not in codebook)\n", col))
          next
        }
        
        col_type <- codebook_col_type(col, cb_types)
        n_unique <- length(unique(historic_data[[col]][!is.na(historic_data[[col]])]))
        
        if(col_type == "Numeric") {
          impute_methods_historic[col] <- "pmm"
          method_counts$pmm <- method_counts$pmm + 1
          cat(sprintf("  %s: pmm (numeric)\n", col))
          
        } else if(col_type == "Categorical") {
          # Ensure it's a factor
          if(!is.factor(historic_data[[col]])) {
            historic_data[[col]] <- as.factor(historic_data[[col]])
          }
          
          # Handle based on cardinality
          if(n_unique <= 10) {
            impute_methods_historic[col] <- "cart"
            method_counts$cart <- method_counts$cart + 1
            cat(sprintf("  %s: cart (%d categories)\n", col, n_unique))
          } else if(n_unique <= 50) {
            impute_methods_historic[col] <- "polyreg"
            method_counts$polyreg <- method_counts$polyreg + 1
            cat(sprintf("  %s: polyreg (%d categories)\n", col, n_unique))
          } else {
            impute_methods_historic[col] <- "sample"
            method_counts$sample <- method_counts$sample + 1
            cat(sprintf("  %s: sample (%d categories - too many for polyreg)\n", col, n_unique))
          }
          
        } else if(col_type == "Boolean") {
          # Ensure it's a factor for logreg
          if(!is.factor(historic_data[[col]])) {
            historic_data[[col]] <- as.factor(historic_data[[col]])
          }
          impute_methods_historic[col] <- "logreg"
          method_counts$logreg <- method_counts$logreg + 1
          cat(sprintf("  %s: logreg (boolean)\n", col))
          
        } else {
          # Fallback for unknown types
          if(is.numeric(historic_data[[col]])) {
            impute_methods_historic[col] <- "pmm"
            method_counts$pmm <- method_counts$pmm + 1
            cat(sprintf("  %s: pmm (numeric fallback)\n", col))
          } else if(is.factor(historic_data[[col]]) || is.character(historic_data[[col]])) {
            if(!is.factor(historic_data[[col]])) {
              historic_data[[col]] <- as.factor(historic_data[[col]])
            }
            if(n_unique <= 10) {
              impute_methods_historic[col] <- "cart"
              method_counts$cart <- method_counts$cart + 1
            } else if(n_unique <= 50) {
              impute_methods_historic[col] <- "polyreg"
              method_counts$polyreg <- method_counts$polyreg + 1
            } else {
              impute_methods_historic[col] <- "sample"
              method_counts$sample <- method_counts$sample + 1
            }
            cat(sprintf("  %s: %s (categorical fallback, %d categories)\n", 
                        col, impute_methods_historic[col], n_unique))
          } else if(is.logical(historic_data[[col]])) {
            if(!is.factor(historic_data[[col]])) {
              historic_data[[col]] <- as.factor(historic_data[[col]])
            }
            impute_methods_historic[col] <- "logreg"
            method_counts$logreg <- method_counts$logreg + 1
            cat(sprintf("  %s: logreg (logical fallback)\n", col))
          }
        }
      }
    }
    
    cat("\n[METHOD SUMMARY]\n")
    cat(sprintf("  PMM: %d variables\n", method_counts$pmm))
    cat(sprintf("  CART: %d variables\n", method_counts$cart))
    cat(sprintf("  PolyReg: %d variables\n", method_counts$polyreg))
    cat(sprintf("  Sample: %d variables\n", method_counts$sample))
    cat(sprintf("  LogReg: %d variables\n", method_counts$logreg))
    flush.console()

    cat("\n[PROGRESS] Starting MICE imputation on historic data...\n")
    cat(sprintf("Timestamp: %s - This may take several minutes\n", Sys.time()))
    cat("Note: Progress will update after each iteration completes\n")
    flush.console()
    
    historic_mice <- mice(historic_data,
                          m = 5,
                          method = impute_methods_historic,
                          predictorMatrix = pred_matrix_historic,
                          ridge = 1e-5,
                          maxit = 5,
                          seed = 550,
                          printFlag = TRUE)
    
    cat("\n[PROGRESS] MICE imputation completed, extracting completed dataset...\n")
    cat(sprintf("Timestamp: %s\n", Sys.time()))
    flush.console()
    
    historic_data <- complete(historic_mice, 1)

    if(length(numeric_missing_historic) > 0) {
      post_stats_historic <- data.frame(
        Variable = numeric_missing_historic,
        Imputed_Mean = sapply(historic_data[numeric_missing_historic], mean, na.rm = TRUE),
        Imputed_Median = sapply(historic_data[numeric_missing_historic], median, na.rm = TRUE),
        Imputed_Variance = sapply(historic_data[numeric_missing_historic], var, na.rm = TRUE),
        stringsAsFactors = FALSE
      )
      comparison_historic <- merge(original_stats_historic, post_stats_historic, by = "Variable")
    }
    
    # Store post-imputation categorical distributions
    if(length(categorical_missing_historic) > 0) {
      post_cat_dist_historic <- list()
      for(col in categorical_missing_historic) {
        post_cat_dist_historic[[col]] <- prop.table(table(historic_data[[col]]))
      }
    }
    
    # Store post-imputation boolean distributions
    if(length(boolean_missing_historic) > 0) {
      post_bool_dist_historic <- list()
      for(col in boolean_missing_historic) {
        post_bool_dist_historic[[col]] <- prop.table(table(historic_data[[col]]))
      }
    }

    cat("Historic data imputation complete\n")
    cat(sprintf("Timestamp: %s\n", Sys.time()))
    flush.console()
  }

  # Impute predict set
  if(remaining_missing_predict > 0) {
    cat("\n--- Processing Predict Set ---\n")
    cat(sprintf("Timestamp: %s\n", Sys.time()))
    flush.console()
    
    cols_with_missing_predict <- names(predict_set)[colSums(is.na(predict_set)) > 0]
    
    cat(sprintf("[PROGRESS] Found %d columns with missing values\n", length(cols_with_missing_predict)))
    flush.console()
    
    # Identify numeric columns with missing values for statistics tracking
    numeric_missing_predict <- cols_with_missing_predict[
      sapply(cols_with_missing_predict, function(col) {
        col %in% names(cb_types$type) && codebook_col_type(col, cb_types) == "Numeric"
      })
    ]
    
    # Identify categorical columns with missing values for distribution tracking
    categorical_missing_predict <- cols_with_missing_predict[
      sapply(cols_with_missing_predict, function(col) {
        col %in% names(cb_types$type) && codebook_col_type(col, cb_types) == "Categorical"
      })
    ]
    
    # Identify boolean columns with missing values for distribution tracking
    boolean_missing_predict <- cols_with_missing_predict[
      sapply(cols_with_missing_predict, function(col) {
        col %in% names(cb_types$type) && codebook_col_type(col, cb_types) == "Boolean"
      })
    ]

    cat(sprintf("[PROGRESS] Numeric: %d, Categorical: %d, Boolean: %d\n", 
                length(numeric_missing_predict), 
                length(categorical_missing_predict),
                length(boolean_missing_predict)))
    flush.console()

    if(length(numeric_missing_predict) > 0) {
      original_stats_predict <- data.frame(
        Variable = numeric_missing_predict,
        Original_Mean = sapply(predict_set[numeric_missing_predict], mean, na.rm = TRUE),
        Original_Median = sapply(predict_set[numeric_missing_predict], median, na.rm = TRUE),
        Original_Variance = sapply(predict_set[numeric_missing_predict], var, na.rm = TRUE),
        stringsAsFactors = FALSE
      )
    }
    
    # Store original categorical distributions
    if(length(categorical_missing_predict) > 0) {
      original_cat_dist_predict <- list()
      for(col in categorical_missing_predict) {
        original_cat_dist_predict[[col]] <- prop.table(table(predict_set[[col]], useNA = "no"))
      }
    }
    
    # Store original boolean distributions
    if(length(boolean_missing_predict) > 0) {
      original_bool_dist_predict <- list()
      for(col in boolean_missing_predict) {
        original_bool_dist_predict[[col]] <- prop.table(table(predict_set[[col]], useNA = "no"))
      }
    }

    cat("Variables with missing values:", length(cols_with_missing_predict), "\n")
    
    # Diagnostic: Analyze imputation complexity
    cat("\n=== Imputation Complexity Analysis (Predict Set) ===\n")
    for(col in cols_with_missing_predict) {
      n_missing <- sum(is.na(predict_set[[col]]))
      n_unique <- length(unique(predict_set[[col]][!is.na(predict_set[[col]])]))
      col_type <- class(predict_set[[col]])[1]
      
      cat(sprintf("%s: %d missing, %d unique values, type=%s\n",
                  col, n_missing, n_unique, col_type))
    }
    flush.console()
    
    cat("\n[PROGRESS] Creating predictor matrix with quickpred()...\n")
    cat(sprintf("Timestamp: %s\n", Sys.time()))
    flush.console()
    
    pred_matrix_predict <- quickpred(predict_set, mincor = 0.3, minpuc = 0.5)
    
    cat("[PROGRESS] Predictor matrix created successfully\n")
    cat(sprintf("Timestamp: %s\n", Sys.time()))
    flush.console()
    
    impute_methods_predict <- make.method(predict_set)

    # Assign imputation methods based on codebook data types
    cat("\n=== Assigning Imputation Methods (Predict Set) ===\n")
    flush.console()
    
    method_counts_predict <- list(pmm = 0, cart = 0, polyreg = 0, sample = 0, logreg = 0)
    
    for(col in names(predict_set)) {
      if(sum(is.na(predict_set[[col]])) > 0) {
        # Skip if column doesn't exist in codebook
        if(!col %in% names(cb_types$type)) {
          cat(sprintf("  %s: skipped (not in codebook)\n", col))
          next
        }
        
        col_type <- codebook_col_type(col, cb_types)
        n_unique <- length(unique(predict_set[[col]][!is.na(predict_set[[col]])]))
        
        if(col_type == "Numeric") {
          impute_methods_predict[col] <- "pmm"
          method_counts_predict$pmm <- method_counts_predict$pmm + 1
          cat(sprintf("  %s: pmm (numeric)\n", col))
          
        } else if(col_type == "Categorical") {
          # Ensure it's a factor
          if(!is.factor(predict_set[[col]])) {
            predict_set[[col]] <- as.factor(predict_set[[col]])
          }
          
          # Handle based on cardinality
          if(n_unique <= 10) {
            impute_methods_predict[col] <- "cart"
            method_counts_predict$cart <- method_counts_predict$cart + 1
            cat(sprintf("  %s: cart (%d categories)\n", col, n_unique))
          } else if(n_unique <= 50) {
            impute_methods_predict[col] <- "polyreg"
            method_counts_predict$polyreg <- method_counts_predict$polyreg + 1
            cat(sprintf("  %s: polyreg (%d categories)\n", col, n_unique))
          } else {
            impute_methods_predict[col] <- "sample"
            method_counts_predict$sample <- method_counts_predict$sample + 1
            cat(sprintf("  %s: sample (%d categories - too many for polyreg)\n", col, n_unique))
          }
          
        } else if(col_type == "Boolean") {
          # Ensure it's a factor for logreg
          if(!is.factor(predict_set[[col]])) {
            predict_set[[col]] <- as.factor(predict_set[[col]])
          }
          impute_methods_predict[col] <- "logreg"
          method_counts_predict$logreg <- method_counts_predict$logreg + 1
          cat(sprintf("  %s: logreg (boolean)\n", col))
          
        } else {
          # Fallback for unknown types
          if(is.numeric(predict_set[[col]])) {
            impute_methods_predict[col] <- "pmm"
            method_counts_predict$pmm <- method_counts_predict$pmm + 1
            cat(sprintf("  %s: pmm (numeric fallback)\n", col))
          } else if(is.factor(predict_set[[col]]) || is.character(predict_set[[col]])) {
            if(!is.factor(predict_set[[col]])) {
              predict_set[[col]] <- as.factor(predict_set[[col]])
            }
            if(n_unique <= 10) {
              impute_methods_predict[col] <- "cart"
              method_counts_predict$cart <- method_counts_predict$cart + 1
            } else if(n_unique <= 50) {
              impute_methods_predict[col] <- "polyreg"
              method_counts_predict$polyreg <- method_counts_predict$polyreg + 1
            } else {
              impute_methods_predict[col] <- "sample"
              method_counts_predict$sample <- method_counts_predict$sample + 1
            }
            cat(sprintf("  %s: %s (categorical fallback, %d categories)\n", 
                        col, impute_methods_predict[col], n_unique))
          } else if(is.logical(predict_set[[col]])) {
            if(!is.factor(predict_set[[col]])) {
              predict_set[[col]] <- as.factor(predict_set[[col]])
            }
            impute_methods_predict[col] <- "logreg"
            method_counts_predict$logreg <- method_counts_predict$logreg + 1
            cat(sprintf("  %s: logreg (logical fallback)\n", col))
          }
        }
      }
    }
    
    cat("\n[METHOD SUMMARY]\n")
    cat(sprintf("  PMM: %d variables\n", method_counts_predict$pmm))
    cat(sprintf("  CART: %d variables\n", method_counts_predict$cart))
    cat(sprintf("  PolyReg: %d variables\n", method_counts_predict$polyreg))
    cat(sprintf("  Sample: %d variables\n", method_counts_predict$sample))
    cat(sprintf("  LogReg: %d variables\n", method_counts_predict$logreg))
    flush.console()

    cat("\n[PROGRESS] Starting MICE imputation on predict set...\n")
    cat(sprintf("Timestamp: %s - This may take several minutes\n", Sys.time()))
    cat("Note: Progress will update after each iteration completes\n")
    flush.console()
    
    predict_mice <- mice(predict_set,
                         m = 5,
                         method = impute_methods_predict,
                         predictorMatrix = pred_matrix_predict,
                         ridge = 1e-5,
                         maxit = 5,
                         seed = 550,
                         printFlag = TRUE)
    
    cat("\n[PROGRESS] MICE imputation completed, extracting completed dataset...\n")
    cat(sprintf("Timestamp: %s\n", Sys.time()))
    flush.console()
    
    predict_set <- complete(predict_mice, 1)

    if(length(numeric_missing_predict) > 0) {
      post_stats_predict <- data.frame(
        Variable = numeric_missing_predict,
        Imputed_Mean = sapply(predict_set[numeric_missing_predict], mean, na.rm = TRUE),
        Imputed_Median = sapply(predict_set[numeric_missing_predict], median, na.rm = TRUE),
        Imputed_Variance = sapply(predict_set[numeric_missing_predict], var, na.rm = TRUE),
        stringsAsFactors = FALSE
      )
      comparison_predict <- merge(original_stats_predict, post_stats_predict, by = "Variable")
    }
    
    # Store post-imputation categorical distributions
    if(length(categorical_missing_predict) > 0) {
      post_cat_dist_predict <- list()
      for(col in categorical_missing_predict) {
        post_cat_dist_predict[[col]] <- prop.table(table(predict_set[[col]]))
      }
    }
    
    # Store post-imputation boolean distributions
    if(length(boolean_missing_predict) > 0) {
      post_bool_dist_predict <- list()
      for(col in boolean_missing_predict) {
        post_bool_dist_predict[[col]] <- prop.table(table(predict_set[[col]]))
      }
    }

    cat("Predict set imputation complete\n")
    cat(sprintf("Timestamp: %s\n", Sys.time()))
    flush.console()
  }

  # Create comparison plots showing percent difference (only for numeric variables)
  cat("\n=== Creating Imputation Comparison Plots ===\n")
  cat(sprintf("Timestamp: %s\n", Sys.time()))
  flush.console()

  if(remaining_missing_historic > 0 && exists("comparison_historic") && nrow(comparison_historic) > 0) {
    comparison_historic_pct <- comparison_historic %>%
      mutate(
        Mean_Pct_Diff = ((Imputed_Mean - Original_Mean) / abs(Original_Mean)) * 100,
        Median_Pct_Diff = ((Imputed_Median - Original_Median) / abs(Original_Median)) * 100,
        Variance_Pct_Diff = ((Imputed_Variance - Original_Variance) / abs(Original_Variance)) * 100
      ) %>%
      select(Variable, Mean_Pct_Diff, Median_Pct_Diff, Variance_Pct_Diff)
    comparison_historic_pct[sapply(comparison_historic_pct, is.infinite)] <- NA
    comparison_long_historic <- comparison_historic_pct %>%
      pivot_longer(cols = -Variable,
                   names_to = "Statistic",
                   names_pattern = "(.*)_Pct_Diff",
                   values_to = "Percent_Difference") %>%
      filter(!is.na(Percent_Difference) & is.finite(Percent_Difference))

    if(nrow(comparison_long_historic) > 0) {
      p_historic <- ggplot(comparison_long_historic,
                           aes(x = Variable, y = Percent_Difference, fill = Statistic)) +
        geom_bar(stat = "identity", position = "dodge") +
        geom_hline(yintercept = 0, linetype = "dashed", color = "black") +
        facet_wrap(~Statistic, scales = "free_y", ncol = 1) +
        theme_minimal() +
        theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
        labs(title = "Historic Data: % Difference in Statistics After Imputation",
             subtitle = "MICE (PMM/CART/PolyReg/Sample/LogReg)",
             x = "Variable",
             y = "Percent Difference (%)",
             fill = "Statistic") +
        scale_fill_manual(values = c("Mean" = "#E69F00", "Median" = "#56B4E9", "Variance" = "#009E73"))

      print(p_historic)
    }
  }

  if(remaining_missing_predict > 0 && exists("comparison_predict") && nrow(comparison_predict) > 0) {
    comparison_predict_pct <- comparison_predict %>%
      mutate(
        Mean_Pct_Diff = ((Imputed_Mean - Original_Mean) / abs(Original_Mean)) * 100,
        Median_Pct_Diff = ((Imputed_Median - Original_Median) / abs(Original_Median)) * 100,
        Variance_Pct_Diff = ((Imputed_Variance - Original_Variance) / abs(Original_Variance)) * 100
      ) %>%
      select(Variable, Mean_Pct_Diff, Median_Pct_Diff, Variance_Pct_Diff)
    comparison_predict_pct[sapply(comparison_predict_pct, is.infinite)] <- NA
    comparison_long_predict <- comparison_predict_pct %>%
      pivot_longer(cols = -Variable,
                   names_to = "Statistic",
                   names_pattern = "(.*)_Pct_Diff",
                   values_to = "Percent_Difference") %>%
      filter(!is.na(Percent_Difference) & is.finite(Percent_Difference))

    if(nrow(comparison_long_predict) > 0) {
      p_predict <- ggplot(comparison_long_predict,
                          aes(x = Variable, y = Percent_Difference, fill = Statistic)) +
        geom_bar(stat = "identity", position = "dodge") +
        geom_hline(yintercept = 0, linetype = "dashed", color = "black") +
        facet_wrap(~Statistic, scales = "free_y", ncol = 1) +
        theme_minimal() +
        theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
        labs(title = "Predict Set: % Difference in Statistics After Imputation",
             subtitle = "MICE (PMM/CART/PolyReg/Sample/LogReg)",
             x = "Variable",
             y = "Percent Difference (%)",
             fill = "Statistic") +
        scale_fill_manual(values = c("Mean" = "#E69F00", "Median" = "#56B4E9", "Variance" = "#009E73"))

      print(p_predict)
    }
  }
  
  # Categorical and Boolean Distribution Comparison
  cat("\n=== Categorical and Boolean Distribution Preservation ===\n")
  
  # Historic Data - Categorical Variables
  if(remaining_missing_historic > 0 && exists("original_cat_dist_historic") && length(original_cat_dist_historic) > 0) {
    cat("\n--- Historic Data: Categorical Variables ---\n")
    
    cat_comparison_historic <- data.frame()
    for(col in names(original_cat_dist_historic)) {
      orig_dist <- original_cat_dist_historic[[col]]
      post_dist <- post_cat_dist_historic[[col]]
      
      # Align categories
      all_categories <- union(names(orig_dist), names(post_dist))
      orig_aligned <- sapply(all_categories, function(cat) {
        ifelse(cat %in% names(orig_dist), orig_dist[cat], 0)
      })
      post_aligned <- sapply(all_categories, function(cat) {
        ifelse(cat %in% names(post_dist), post_dist[cat], 0)
      })
      
      # Calculate absolute difference in proportions
      for(i in seq_along(all_categories)) {
        cat_comparison_historic <- rbind(cat_comparison_historic, data.frame(
          Variable = col,
          Category = all_categories[i],
          Original_Prop = orig_aligned[i],
          Imputed_Prop = post_aligned[i],
          Abs_Diff = abs(post_aligned[i] - orig_aligned[i]),
          stringsAsFactors = FALSE
        ))
      }
    }
    
    # Print summary statistics
    cat("Maximum absolute difference in category proportions:", 
        sprintf("%.4f", max(cat_comparison_historic$Abs_Diff)), "\n")
    cat("Mean absolute difference in category proportions:", 
        sprintf("%.4f", mean(cat_comparison_historic$Abs_Diff)), "\n")
    
    # Create visualization (only for variables with ≤10 categories for readability)
    cat_vars_to_plot <- names(original_cat_dist_historic)[
      sapply(names(original_cat_dist_historic), function(v) {
        length(unique(historic_data[[v]])) <= 10
      })
    ]
    
    if(length(cat_vars_to_plot) > 0) {
      cat_comparison_long_historic <- cat_comparison_historic %>%
        filter(Variable %in% cat_vars_to_plot) %>%
        pivot_longer(cols = c(Original_Prop, Imputed_Prop),
                     names_to = "Distribution",
                     values_to = "Proportion") %>%
        mutate(Distribution = ifelse(Distribution == "Original_Prop", "Pre-Imputation", "Post-Imputation"))
      
      p_cat_historic <- ggplot(cat_comparison_long_historic,
                               aes(x = Category, y = Proportion, fill = Distribution)) +
        geom_bar(stat = "identity", position = "dodge") +
        facet_wrap(~Variable, scales = "free_x", ncol = 2) +
        theme_minimal() +
        theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 8),
              legend.position = "bottom") +
        labs(title = "Historic Data: Categorical Distribution Preservation",
             subtitle = "Comparison of category proportions (low-cardinality variables only)",
             x = "Category",
             y = "Proportion",
             fill = "") +
        scale_fill_manual(values = c("Pre-Imputation" = "#0072B2", "Post-Imputation" = "#D55E00"))
      
      print(p_cat_historic)
    }
  }
  
  # Historic Data - Boolean Variables
  if(remaining_missing_historic > 0 && exists("original_bool_dist_historic") && length(original_bool_dist_historic) > 0) {
    cat("\n--- Historic Data: Boolean Variables ---\n")
    
    bool_comparison_historic <- data.frame()
    for(col in names(original_bool_dist_historic)) {
      orig_dist <- original_bool_dist_historic[[col]]
      post_dist <- post_bool_dist_historic[[col]]
      
      # Align categories
      all_categories <- union(names(orig_dist), names(post_dist))
      orig_aligned <- sapply(all_categories, function(cat) {
        ifelse(cat %in% names(orig_dist), orig_dist[cat], 0)
      })
      post_aligned <- sapply(all_categories, function(cat) {
        ifelse(cat %in% names(post_dist), post_dist[cat], 0)
      })
      
      # Calculate absolute difference in proportions
      for(i in seq_along(all_categories)) {
        bool_comparison_historic <- rbind(bool_comparison_historic, data.frame(
          Variable = col,
          Category = all_categories[i],
          Original_Prop = orig_aligned[i],
          Imputed_Prop = post_aligned[i],
          Abs_Diff = abs(post_aligned[i] - orig_aligned[i]),
          stringsAsFactors = FALSE
        ))
      }
    }
    
    # Print summary statistics
    cat("Maximum absolute difference in boolean proportions:", 
        sprintf("%.4f", max(bool_comparison_historic$Abs_Diff)), "\n")
    cat("Mean absolute difference in boolean proportions:", 
        sprintf("%.4f", mean(bool_comparison_historic$Abs_Diff)), "\n")
    
    # Create visualization
    bool_comparison_long_historic <- bool_comparison_historic %>%
      pivot_longer(cols = c(Original_Prop, Imputed_Prop),
                   names_to = "Distribution",
                   values_to = "Proportion") %>%
      mutate(Distribution = ifelse(Distribution == "Original_Prop", "Pre-Imputation", "Post-Imputation"))
    
    p_bool_historic <- ggplot(bool_comparison_long_historic,
                              aes(x = Category, y = Proportion, fill = Distribution)) +
      geom_bar(stat = "identity", position = "dodge") +
      facet_wrap(~Variable, scales = "free_x", ncol = 2) +
      theme_minimal() +
      theme(axis.text.x = element_text(angle = 45, hjust = 1),
            legend.position = "bottom") +
      labs(title = "Historic Data: Boolean Distribution Preservation (LogReg)",
           subtitle = "Comparison of TRUE/FALSE proportions before and after MICE imputation",
           x = "Value",
           y = "Proportion",
           fill = "") +
      scale_fill_manual(values = c("Pre-Imputation" = "#0072B2", "Post-Imputation" = "#D55E00"))
    
    print(p_bool_historic)
  }
  
  # Predict Set - Categorical Variables
  if(remaining_missing_predict > 0 && exists("original_cat_dist_predict") && length(original_cat_dist_predict) > 0) {
    cat("\n--- Predict Set: Categorical Variables ---\n")
    
    cat_comparison_predict <- data.frame()
    for(col in names(original_cat_dist_predict)) {
      orig_dist <- original_cat_dist_predict[[col]]
      post_dist <- post_cat_dist_predict[[col]]
      
      # Align categories
      all_categories <- union(names(orig_dist), names(post_dist))
      orig_aligned <- sapply(all_categories, function(cat) {
        ifelse(cat %in% names(orig_dist), orig_dist[cat], 0)
      })
      post_aligned <- sapply(all_categories, function(cat) {
        ifelse(cat %in% names(post_dist), post_dist[cat], 0)
      })
      
      # Calculate absolute difference in proportions
      for(i in seq_along(all_categories)) {
        cat_comparison_predict <- rbind(cat_comparison_predict, data.frame(
          Variable = col,
          Category = all_categories[i],
          Original_Prop = orig_aligned[i],
          Imputed_Prop = post_aligned[i],
          Abs_Diff = abs(post_aligned[i] - orig_aligned[i]),
          stringsAsFactors = FALSE
        ))
      }
    }
    
    # Print summary statistics
    cat("Maximum absolute difference in category proportions:", 
        sprintf("%.4f", max(cat_comparison_predict$Abs_Diff)), "\n")
    cat("Mean absolute difference in category proportions:", 
        sprintf("%.4f", mean(cat_comparison_predict$Abs_Diff)), "\n")
    
    # Create visualization (only for variables with ≤10 categories for readability)
    cat_vars_to_plot <- names(original_cat_dist_predict)[
      sapply(names(original_cat_dist_predict), function(v) {
        length(unique(predict_set[[v]])) <= 10
      })
    ]
    
    if(length(cat_vars_to_plot) > 0) {
      cat_comparison_long_predict <- cat_comparison_predict %>%
        filter(Variable %in% cat_vars_to_plot) %>%
        pivot_longer(cols = c(Original_Prop, Imputed_Prop),
                     names_to = "Distribution",
                     values_to = "Proportion") %>%
        mutate(Distribution = ifelse(Distribution == "Original_Prop", "Pre-Imputation", "Post-Imputation"))
      
      p_cat_predict <- ggplot(cat_comparison_long_predict,
                              aes(x = Category, y = Proportion, fill = Distribution)) +
        geom_bar(stat = "identity", position = "dodge") +
        facet_wrap(~Variable, scales = "free_x", ncol = 2) +
        theme_minimal() +
        theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 8),
              legend.position = "bottom") +
        labs(title = "Predict Set: Categorical Distribution Preservation",
             subtitle = "Comparison of category proportions (low-cardinality variables only)",
             x = "Category",
             y = "Proportion",
             fill = "") +
        scale_fill_manual(values = c("Pre-Imputation" = "#0072B2", "Post-Imputation" = "#D55E00"))
      
      print(p_cat_predict)
    }
  }
  
  # Predict Set - Boolean Variables
  if(remaining_missing_predict > 0 && exists("original_bool_dist_predict") && length(original_bool_dist_predict) > 0) {
    cat("\n--- Predict Set: Boolean Variables ---\n")
    
    bool_comparison_predict <- data.frame()
    for(col in names(original_bool_dist_predict)) {
      orig_dist <- original_bool_dist_predict[[col]]
      post_dist <- post_bool_dist_predict[[col]]
      
      # Align categories
      all_categories <- union(names(orig_dist), names(post_dist))
      orig_aligned <- sapply(all_categories, function(cat) {
        ifelse(cat %in% names(orig_dist), orig_dist[cat], 0)
      })
      post_aligned <- sapply(all_categories, function(cat) {
        ifelse(cat %in% names(post_dist), post_dist[cat], 0)
      })
      
      # Calculate absolute difference in proportions
      for(i in seq_along(all_categories)) {
        bool_comparison_predict <- rbind(bool_comparison_predict, data.frame(
          Variable = col,
          Category = all_categories[i],
          Original_Prop = orig_aligned[i],
          Imputed_Prop = post_aligned[i],
          Abs_Diff = abs(post_aligned[i] - orig_aligned[i]),
          stringsAsFactors = FALSE
        ))
      }
    }
    
    # Print summary statistics
    cat("Maximum absolute difference in boolean proportions:", 
        sprintf("%.4f", max(bool_comparison_predict$Abs_Diff)), "\n")
    cat("Mean absolute difference in boolean proportions:", 
        sprintf("%.4f", mean(bool_comparison_predict$Abs_Diff)), "\n")
    
    # Create visualization
    bool_comparison_long_predict <- bool_comparison_predict %>%
      pivot_longer(cols = c(Original_Prop, Imputed_Prop),
                   names_to = "Distribution",
                   values_to = "Proportion") %>%
      mutate(Distribution = ifelse(Distribution == "Original_Prop", "Pre-Imputation", "Post-Imputation"))
    
    p_bool_predict <- ggplot(bool_comparison_long_predict,
                             aes(x = Category, y = Proportion, fill = Distribution)) +
      geom_bar(stat = "identity", position = "dodge") +
      facet_wrap(~Variable, scales = "free_x", ncol = 2) +
      theme_minimal() +
      theme(axis.text.x = element_text(angle = 45, hjust = 1),
            legend.position = "bottom") +
      labs(title = "Predict Set: Boolean Distribution Preservation (LogReg)",
           subtitle = "Comparison of TRUE/FALSE proportions before and after MICE imputation",
           x = "Value",
           y = "Proportion",
           fill = "") +
      scale_fill_manual(values = c("Pre-Imputation" = "#0072B2", "Post-Imputation" = "#D55E00"))
    
    print(p_bool_predict)
  }

  # Final verification
  cat("\n=== Post-Imputation Verification ===\n")
  final_missing_historic <- sum(is.na(historic_data))
  final_missing_predict <- sum(is.na(predict_set))

  cat("Missing values in historic data:", final_missing_historic, "\n")
  cat("Missing values in predict set:", final_missing_predict, "\n")
}


[CHECKPOINT] About to check if imputation needed...

=== Imputing Missing Values with MICE ===
Timestamp: 2025-11-25 17:13:18.459939

--- Processing Historic Data ---
Timestamp: 2025-11-25 17:13:18.46046
[PROGRESS] Found 39 columns with missing values
[PROGRESS] Numeric: 10, Categorical: 24, Boolean: 5
Variables with missing values: 39 

=== Imputation Complexity Analysis (Historic Data) ===
char_ext_wall: 26 missing, 4 unique values, type=factor
char_roof_cnst: 26 missing, 6 unique values, type=factor
char_bsmt: 25 missing, 4 unique values, type=factor
char_bsmt_fin: 25 missing, 3 unique values, type=factor
char_heat: 26 missing, 4 unique values, type=factor
char_oheat: 172 missing, 2 unique values, type=factor
char_air: 23 missing, 2 unique values, type=factor
char_frpl: 25 missing, 10 unique values, type=numeric
char_attic_type: 26 missing, 3 unique values, type=factor
char_tp_plan: 14394 missing, 2 unique values, type=factor
char_cnst_qlty: 57 missing, 2 unique values, type=factor

---
### Exploratory Data Analysis

In [ ]:
# =====================================================
# ENHANCED VARIABLE CLASSIFICATION
# =====================================================

cat("\n=== Classifying Variables by Data Type ===\n")

# Get codebook types
cb_types <- get_codebook_types()

# Classification function with granular types
classify_variable <- function(var, data, cb_types) {
  # Get codebook info
  cb_type <- codebook_col_type(var, cb_types)
  
  # Get R data type
  r_type <- class(data[[var]])[1]
  
  # Determine granular classification
  if(cb_type == "Numeric") {
    # Check if it's integer-like (whole numbers, counts) or continuous
    if(r_type %in% c("integer", "factor")) {
      return("Integer")
    } else if(is.numeric(data[[var]])) {
      # Check if values are mostly whole numbers (counts/discrete)
      non_na_vals <- data[[var]][!is.na(data[[var]])]
      if(length(non_na_vals) > 0) {
        whole_number_pct <- mean(non_na_vals == floor(non_na_vals))
        # If variable name suggests count (beds, rooms, baths) or >95% whole numbers
        is_count <- grepl("beds|rooms|bath|apts|frpl", var, ignore.case = TRUE)
        if(is_count || whole_number_pct > 0.95) {
          return("Integer")
        } else {
          return("Continuous")
        }
      }
      return("Continuous")
    }
  } else if(cb_type == "Categorical") {
    # Check if ordinal (quality, condition, size ratings)
    is_ordinal <- grepl("qlty|quality|cnd|condition|size|rating|grade", var, ignore.case = TRUE)
    if(is_ordinal) {
      return("Ordinal")
    } else {
      return("Nominal")
    }
  } else if(cb_type == "Boolean") {
    return("Boolean")
  }
  
  return("Unknown")
}

# Classify all variables
variable_classifications <- data.frame(
  Variable = names(historic_data),
  Classification = sapply(names(historic_data), function(v) {
    classify_variable(v, historic_data, cb_types)
  }),
  stringsAsFactors = FALSE
)

# Group variables by classification
integer_vars <- variable_classifications %>% filter(Classification == "Integer") %>% pull(Variable)
continuous_vars <- variable_classifications %>% filter(Classification == "Continuous") %>% pull(Variable)
ordinal_vars <- variable_classifications %>% filter(Classification == "Ordinal") %>% pull(Variable)
nominal_vars <- variable_classifications %>% filter(Classification == "Nominal") %>% pull(Variable)
boolean_vars <- variable_classifications %>% filter(Classification == "Boolean") %>% pull(Variable)

cat("\nVariable Classification Summary:\n")
cat("  Integer Variables:", length(integer_vars), "\n")
cat("  Continuous Variables:", length(continuous_vars), "\n")
cat("  Ordinal Variables:", length(ordinal_vars), "\n")
cat("  Nominal Variables:", length(nominal_vars), "\n")
cat("  Boolean Variables:", length(boolean_vars), "\n")

In [ ]:
# =====================================================
# SUMMARY STATISTICS BY CLASSIFICATION
# =====================================================

cat("\n=== Summary Statistics ===\n\n")

# Integer variables
if(length(integer_vars) > 0) {
  cat("--- Integer Variables ---\n")
  integer_stats <- historic_data %>%
    select(all_of(integer_vars)) %>%
    summarise(across(everything(), 
                     list(mean = ~mean(.x, na.rm = TRUE),
                          median = ~median(.x, na.rm = TRUE),
                          sd = ~sd(.x, na.rm = TRUE),
                          variance = ~var(.x, na.rm = TRUE),
                          min = ~min(.x, na.rm = TRUE),
                          max = ~max(.x, na.rm = TRUE)),
                     .names = "{.col}_{.fn}")) %>%
    pivot_longer(everything(),
                 names_to = c("Variable", "Statistic"),
                 names_pattern = "(.*)_(mean|median|sd|variance|min|max)",
                 values_to = "Value") %>%
    pivot_wider(names_from = Statistic, values_from = Value)
  
  print(integer_stats, n = Inf)
}

# Continuous variables
if(length(continuous_vars) > 0) {
  cat("\n--- Continuous Variables ---\n")
  continuous_stats <- historic_data %>%
    select(all_of(continuous_vars)) %>%
    summarise(across(everything(), 
                     list(mean = ~mean(.x, na.rm = TRUE),
                          median = ~median(.x, na.rm = TRUE),
                          sd = ~sd(.x, na.rm = TRUE),
                          variance = ~var(.x, na.rm = TRUE),
                          min = ~min(.x, na.rm = TRUE),
                          max = ~max(.x, na.rm = TRUE)),
                     .names = "{.col}_{.fn}")) %>%
    pivot_longer(everything(),
                 names_to = c("Variable", "Statistic"),
                 names_pattern = "(.*)_(mean|median|sd|variance|min|max)",
                 values_to = "Value") %>%
    pivot_wider(names_from = Statistic, values_from = Value)
  
  print(continuous_stats, n = Inf)
}

# Ordinal variables - NUMERIC TREATMENT
if(length(ordinal_vars) > 0) {
  cat("\n--- Ordinal Variables (Numeric Statistics) ---\n")
  
  # Convert ordinal to numeric for statistics
  ordinal_numeric <- historic_data %>%
    select(all_of(ordinal_vars)) %>%
    mutate(across(everything(), ~as.numeric(as.factor(.x))))
  
  ordinal_stats <- ordinal_numeric %>%
    summarise(across(everything(), 
                     list(mean = ~mean(.x, na.rm = TRUE),
                          median = ~median(.x, na.rm = TRUE),
                          sd = ~sd(.x, na.rm = TRUE),
                          variance = ~var(.x, na.rm = TRUE),
                          min = ~min(.x, na.rm = TRUE),
                          max = ~max(.x, na.rm = TRUE)),
                     .names = "{.col}_{.fn}")) %>%
    pivot_longer(everything(),
                 names_to = c("Variable", "Statistic"),
                 names_pattern = "(.*)_(mean|median|sd|variance|min|max)",
                 values_to = "Value") %>%
    pivot_wider(names_from = Statistic, values_from = Value)
  
  print(ordinal_stats, n = Inf)
  
  cat("\n--- Ordinal Variables (Category Distributions) ---\n")
  for(var in head(ordinal_vars, 8)) {
    cat("\n", var, ":\n", sep = "")
    freq_table <- table(historic_data[[var]], useNA = "no")
    freq_pct <- prop.table(freq_table) * 100
    
    for(i in seq_along(freq_pct)) {
      cat(sprintf("  %s: %d (%.1f%%)\n", names(freq_pct)[i], freq_table[i], freq_pct[i]))
    }
  }
}

# Nominal summaries
if(length(nominal_vars) > 0) {
  cat("\n--- Nominal Variables (Top Categories) ---\n")
  for(var in head(nominal_vars, 5)) {
    cat("\n", var, ":\n", sep = "")
    freq_table <- table(historic_data[[var]], useNA = "no")
    cat("  Unique values:", length(freq_table), "\n")
    freq_pct <- prop.table(freq_table) * 100
    top_3 <- head(sort(freq_pct, decreasing = TRUE), 3)
    for(i in seq_along(top_3)) {
      cat(sprintf("  %s: %d (%.1f%%)\n", names(top_3)[i], freq_table[names(top_3)[i]], top_3[i]))
    }
  }
}

# Boolean summaries
if(length(boolean_vars) > 0) {
  cat("\n--- Boolean Variables ---\n")
  for(var in boolean_vars) {
    if(var %in% names(historic_data)) {
      true_count <- sum(historic_data[[var]], na.rm = TRUE)
      total_count <- sum(!is.na(historic_data[[var]]))
      true_pct <- (true_count / total_count) * 100
      cat(sprintf("  %s: TRUE=%d (%.1f%%), FALSE=%d (%.1f%%)\n",
                  var, true_count, true_pct, total_count - true_count, 100 - true_pct))
    }
  }
}

In [ ]:
# =====================================================
# INTEGER VARIABLES: HISTOGRAMS
# =====================================================

cat("\n=== Integer Variable Distributions (Histograms) ===\n")

if(length(integer_vars) > 0) {
  
  for(var in integer_vars[1:min(8, length(integer_vars))]) {
    
    p <- ggplot(historic_data, aes(x = .data[[var]])) +
      geom_histogram(bins = 10, fill = "#2E86AB", color = "#023E73", alpha = 0.85) +
      geom_vline(aes(xintercept = mean(.data[[var]], na.rm = TRUE)),
                 color = "#E63946", linetype = "dashed", size = 1.2) +
      geom_vline(aes(xintercept = median(.data[[var]], na.rm = TRUE)),
                 color = "#06FFA5", linetype = "dashed", size = 1.2) +
      theme_minimal() +
      theme(
        plot.title = element_text(size = 13, face = "bold", hjust = 0.5),
        plot.subtitle = element_text(size = 9, hjust = 0.5, color = "gray30"),
        axis.title = element_text(size = 10, face = "bold"),
        axis.text.x = element_text(size = 9),
        panel.grid.minor = element_blank()
      ) +
      labs(
        title = paste("Distribution:", var),
        subtitle = sprintf("Mean=%.1f | Median=%.1f | SD=%.1f (Integer)",
                          mean(historic_data[[var]], na.rm = TRUE),
                          median(historic_data[[var]], na.rm = TRUE),
                          sd(historic_data[[var]], na.rm = TRUE)),
        x = var,
        y = "Frequency"
      )
    
    print(p)
  }
}

In [ ]:
# =====================================================
# ORDINAL VARIABLES: HISTOGRAMS
# =====================================================

cat("\n=== Ordinal Variable Distributions (Histograms) ===\n")

if(length(ordinal_vars) > 0) {
  
  for(var in ordinal_vars[1:min(8, length(ordinal_vars))]) {
    
    freq_data <- as.data.frame(table(historic_data[[var]])) %>%
      arrange(Var1)
    
    colnames(freq_data) <- c("Category", "Count")
    
    # Calculate numeric stats for subtitle
    numeric_vals <- as.numeric(as.factor(historic_data[[var]]))
    mean_val <- mean(numeric_vals, na.rm = TRUE)
    median_val <- median(numeric_vals, na.rm = TRUE)
    sd_val <- sd(numeric_vals, na.rm = TRUE)
    
    p <- ggplot(freq_data, aes(x = Category, y = Count)) +
      geom_bar(stat = "identity", fill = "#6A4C93", color = "#1A1423", alpha = 0.85) +
      geom_text(aes(label = Count), vjust = -0.5, size = 3.5, fontface = "bold") +
      theme_minimal() +
      theme(
        plot.title = element_text(size = 13, face = "bold", hjust = 0.5),
        plot.subtitle = element_text(size = 9, hjust = 0.5, color = "gray30"),
        axis.title = element_text(size = 10, face = "bold"),
        axis.text.x = element_text(angle = 45, hjust = 1, size = 9)
      ) +
      labs(
        title = paste("Distribution:", var),
        subtitle = sprintf("Mean=%.2f | Median=%.2f | SD=%.2f (Ordinal)", 
                          mean_val, median_val, sd_val),
        x = "Category",
        y = "Frequency"
      )
    
    print(p)
  }
}

In [ ]:
# =====================================================
# CONTINUOUS VARIABLES: BOX & WHISKER PLOTS
# =====================================================

cat("\n=== Continuous Variable Distributions (Box & Whisker Plots) ===\n")

if(length(continuous_vars) > 0) {
  
  for(var in continuous_vars[1:min(10, length(continuous_vars))]) {
    
    # Calculate outlier boundaries
    Q1 <- quantile(historic_data[[var]], 0.25, na.rm = TRUE)
    Q3 <- quantile(historic_data[[var]], 0.75, na.rm = TRUE)
    IQR_val <- Q3 - Q1
    lower_bound <- Q1 - 1.5 * IQR_val
    upper_bound <- Q3 + 1.5 * IQR_val
    
    # Identify outliers
    outliers <- historic_data %>%
      filter(!is.na(.data[[var]])) %>%
      filter(.data[[var]] < lower_bound | .data[[var]] > upper_bound) %>%
      select(!!sym(var))
    
    n_outliers <- nrow(outliers)
    
    p <- ggplot(historic_data, aes(y = .data[[var]])) +
      geom_boxplot(aes(x = ""), 
                   fill = "#4ECDC4", 
                   color = "#023E73", 
                   alpha = 0.7,
                   outlier.colour = "#E63946",
                   outlier.shape = 16,
                   outlier.size = 2,
                   outlier.alpha = 0.6,
                   width = 0.5) +
      stat_boxplot(aes(x = ""), geom = "errorbar", width = 0.3, color = "#023E73") +
      theme_minimal() +
      theme(
        plot.title = element_text(size = 13, face = "bold", hjust = 0.5),
        plot.subtitle = element_text(size = 9, hjust = 0.5, color = "gray30"),
        axis.title = element_text(size = 10, face = "bold"),
        axis.text = element_text(size = 9),
        axis.title.x = element_blank(),
        axis.text.x = element_blank(),
        panel.grid.major.x = element_blank()
      ) +
      labs(
        title = paste("Distribution:", var),
        subtitle = sprintf("Median=%.2f | IQR=%.2f | Outliers=%d (Continuous)",
                          median(historic_data[[var]], na.rm = TRUE),
                          IQR_val, n_outliers),
        y = var
      ) +
      scale_y_continuous(labels = scales::comma)
    
    print(p)
  }
  
  # Multi-panel box plot overview
  cat("\n--- Multi-Panel Box Plot Overview ---\n")
  
  continuous_sample_vars <- continuous_vars[1:min(9, length(continuous_vars))]
  
  if(length(continuous_sample_vars) >= 3) {
    
    plot_data_long <- historic_data %>%
      select(all_of(continuous_sample_vars)) %>%
      pivot_longer(everything(), names_to = "Variable", values_to = "Value")
    
    p_multi <- ggplot(plot_data_long, aes(x = "", y = Value)) +
      geom_boxplot(fill = "#38B000", 
                   color = "#1A5E00", 
                   alpha = 0.7,
                   outlier.colour = "#E63946",
                   outlier.shape = 16,
                   outlier.size = 1,
                   outlier.alpha = 0.5) +
      facet_wrap(~Variable, scales = "free_y", ncol = 3) +
      theme_minimal() +
      theme(
        plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
        strip.text = element_text(size = 9, face = "bold"),
        axis.text.y = element_text(size = 7),
        axis.text.x = element_blank(),
        axis.title = element_text(size = 9),
        axis.title.x = element_blank(),
        panel.grid.major.x = element_blank()
      ) +
      labs(
        title = "Continuous Variables - Box & Whisker Plot Overview",
        y = "Value"
      ) +
      scale_y_continuous(labels = scales::comma)
    
    print(p_multi)
  }
  
  # Horizontal box plots for comparison
  cat("\n--- Horizontal Comparison Box Plots ---\n")
  
  if(length(continuous_sample_vars) >= 3) {
    
    # Normalize data for better comparison
    plot_data_long_scaled <- historic_data %>%
      select(all_of(continuous_sample_vars)) %>%
      mutate(across(everything(), ~scale(.x)[,1])) %>%
      pivot_longer(everything(), names_to = "Variable", values_to = "Scaled_Value")
    
    p_horizontal <- ggplot(plot_data_long_scaled, 
                           aes(x = reorder(Variable, Scaled_Value, FUN = median), 
                               y = Scaled_Value)) +
      geom_boxplot(fill = "#FF6B6B", 
                   color = "#C92A2A", 
                   alpha = 0.7,
                   outlier.colour = "#E63946",
                   outlier.shape = 16,
                   outlier.size = 1.5,
                   outlier.alpha = 0.6) +
      coord_flip() +
      theme_minimal() +
      theme(
        plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
        plot.subtitle = element_text(size = 9, hjust = 0.5, color = "gray30"),
        axis.title = element_text(size = 10, face = "bold"),
        axis.text = element_text(size = 9)
      ) +
      labs(
        title = "Continuous Variables - Standardized Comparison",
        subtitle = "Scaled to mean=0, SD=1 for comparison",
        x = "Variable",
        y = "Standardized Value"
      ) +
      geom_hline(yintercept = 0, linetype = "dashed", color = "gray40", size = 0.8)
    
    print(p_horizontal)
  }
}

In [ ]:
# =====================================================
# NOMINAL VARIABLES: PIE CHARTS
# =====================================================

cat("\n=== Nominal Variable Distributions (Pie Charts) ===\n")

if(length(nominal_vars) > 0) {
  
  for(var in nominal_vars[1:min(8, length(nominal_vars))]) {
    
    # Get frequency data (top 8 categories + "Other")
    freq_table <- table(historic_data[[var]], useNA = "no")
    freq_df <- as.data.frame(freq_table) %>%
      arrange(desc(Freq))
    
    colnames(freq_df) <- c("Category", "Count")
    
    # Keep top 7, group rest as "Other"
    if(nrow(freq_df) > 7) {
      top_cats <- freq_df[1:7, ]
      other_count <- sum(freq_df$Count[8:nrow(freq_df)])
      freq_df <- rbind(top_cats, data.frame(Category = "Other", Count = other_count))
    }
    
    freq_df <- freq_df %>%
      mutate(
        Percentage = round(Count / sum(Count) * 100, 1),
        Label = ifelse(Percentage >= 3, paste0(Percentage, "%"), "")
      )
    
    p_pie <- ggplot(freq_df, aes(x = "", y = Count, fill = Category)) +
      geom_bar(stat = "identity", width = 1, color = "white", size = 1.5) +
      coord_polar("y", start = 0) +
      geom_text(aes(label = Label), 
                position = position_stack(vjust = 0.5),
                size = 3.5, fontface = "bold", color = "black") +
      theme_void() +
      theme(
        plot.title = element_text(size = 13, face = "bold", hjust = 0.5, margin = margin(b = 10)),
        plot.subtitle = element_text(size = 9, hjust = 0.5, color = "gray40", margin = margin(b = 15)),
        legend.position = "right",
        legend.title = element_text(size = 10, face = "bold"),
        legend.text = element_text(size = 8)
      ) +
      labs(
        title = paste("Distribution:", var),
        subtitle = paste("(Nominal -", length(freq_table), "unique values)"),
        fill = "Category"
      ) +
      scale_fill_brewer(palette = "Set3")
    
    print(p_pie)
  }
}

In [ ]:
# =====================================================
# BOOLEAN VARIABLES: BAR CHARTS
# =====================================================

cat("\n=== Boolean Variable Distributions (Bar Charts) ===\n")

if(length(boolean_vars) > 0) {
  
  for(var in boolean_vars) {
    
    if(var %in% names(historic_data)) {
      
      freq_data <- data.frame(
        Value = c("TRUE", "FALSE"),
        Count = c(
          sum(historic_data[[var]], na.rm = TRUE),
          sum(!historic_data[[var]], na.rm = TRUE)
        )
      ) %>%
        mutate(
          Percentage = round(Count / sum(Count) * 100, 1),
          Label = paste0(Count, "\n(", Percentage, "%)")
        )
      
      p_bar <- ggplot(freq_data, aes(x = Value, y = Count, fill = Value)) +
        geom_bar(stat = "identity", color = "black", alpha = 0.85, width = 0.6) +
        geom_text(aes(label = Label), vjust = -0.5, size = 4, fontface = "bold") +
        theme_minimal() +
        theme(
          plot.title = element_text(size = 13, face = "bold", hjust = 0.5),
          plot.subtitle = element_text(size = 9, hjust = 0.5, color = "gray30"),
          axis.title = element_text(size = 10, face = "bold"),
          axis.text = element_text(size = 10),
          legend.position = "none",
          panel.grid.major.x = element_blank()
        ) +
        labs(
          title = paste("Distribution:", var),
          subtitle = "(Boolean Variable)",
          x = "Value",
          y = "Count"
        ) +
        scale_fill_manual(values = c("TRUE" = "#4CAF50", "FALSE" = "#F44336")) +
        scale_y_continuous(expand = expansion(mult = c(0, 0.15)))
      
      print(p_bar)
    }
  }
}

In [ ]:
# =====================================================
# CORRELATION ANALYSIS
# =====================================================

cat("\n=== Correlation Analysis ===\n")

# Combine integer, continuous, and ordinal (as numeric) for correlation
numeric_for_corr <- c(integer_vars, continuous_vars)
numeric_for_corr <- intersect(numeric_for_corr, names(historic_data))

# Add ordinal as numeric
if(length(ordinal_vars) > 0) {
  ordinal_in_data <- intersect(ordinal_vars, names(historic_data))
  numeric_for_corr <- c(numeric_for_corr, ordinal_in_data)
}

if(length(numeric_for_corr) >= 2) {
  
  # Prepare data - convert ordinal to numeric
  cor_data <- historic_data %>%
    select(all_of(numeric_for_corr))
  
  # Convert ordinal columns to numeric
  for(col in ordinal_in_data) {
    if(col %in% names(cor_data)) {
      cor_data[[col]] <- as.numeric(as.factor(cor_data[[col]]))
    }
  }
  
  # Keep only numeric columns
  cor_data <- cor_data %>%
    select(where(is.numeric))
  
  # Remove zero-variance columns
  cor_data <- cor_data %>%
    select(where(~var(.x, na.rm = TRUE) > 0))
  
  # Calculate correlation matrix
  cor_matrix <- cor(cor_data, use = "pairwise.complete.obs")
  
  # Full correlation heatmap
  cat("\n--- Creating Correlation Heatmap ---\n")
  
  corrplot(cor_matrix, 
           method = "color",
           type = "upper",
           order = "hclust",
           tl.col = "black",
           tl.srt = 45,
           tl.cex = 0.7,
           cl.cex = 0.8,
           col = colorRampPalette(c("#D32F2F", "#FFFFFF", "#1976D2"))(200),
           title = "Correlation Matrix - Numeric Variables (incl. Ordinal)",
           mar = c(0, 0, 2, 0))
  
  # Sale price correlations
  if("sale_price" %in% colnames(cor_matrix)) {
    cat("\n--- Top Correlations with Sale Price ---\n")
    
    price_cors <- cor_matrix["sale_price", ]
    price_cors <- price_cors[order(abs(price_cors), decreasing = TRUE)]
    price_cors <- price_cors[names(price_cors) != "sale_price"]
    
    top_15 <- head(price_cors, 15)
    for(i in seq_along(top_15)) {
      cat(sprintf("  %s: %.3f\n", names(top_15)[i], top_15[i]))
    }
    
    # Bar chart of correlations
    price_cor_df <- data.frame(
      Variable = names(top_15),
      Correlation = as.numeric(top_15)
    )
    
    p_cor <- ggplot(price_cor_df, aes(x = reorder(Variable, abs(Correlation)), 
                                       y = Correlation, fill = Correlation > 0)) +
      geom_bar(stat = "identity", color = "black", alpha = 0.85) +
      geom_text(aes(label = sprintf("%.2f", Correlation)),
                hjust = ifelse(price_cor_df$Correlation > 0, -0.2, 1.2),
                size = 3.5, fontface = "bold") +
      coord_flip() +
      theme_minimal() +
      theme(
        plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
        axis.title = element_text(size = 11, face = "bold"),
        legend.position = "none"
      ) +
      labs(
        title = "Top 15 Correlations with Sale Price",
        x = "Variable",
        y = "Correlation Coefficient"
      ) +
      scale_fill_manual(values = c("TRUE" = "#2E7D32", "FALSE" = "#C62828")) +
      geom_hline(yintercept = 0, linetype = "solid", color = "black", size = 0.8)
    
    print(p_cor)
  }
}

cat("\n=== EDA Complete ===\n")
cat("\nVisualization Summary:")
cat("\n  - Integer: Histograms")
cat("\n  - Continuous: Box & Whisker Plots (red outliers)")
cat("\n  - Ordinal: Numeric stats + Histograms")
cat("\n  - Nominal: Pie Charts")
cat("\n  - Boolean: Bar Charts")
cat("\n  - Correlation: Heatmap (Integer + Continuous + Ordinal)\n\n")